# Messung der Laufzeien parallelisierter Programme mit OpenMP
## Beispiel: Addition von 2 Vektoren
In diesem Notebook wollen wir ihnen zeigen, wie sie mit Hilfe der von uns Entwickelten Werkzeuge zur Zeit-Messung eines mit OpenMP parallelisierten Programmes die Leistung und den Speedup eines parallelen Programms darstellen können. Wir verwenden dazu das Beispiel der Vektoraddition aus dem Skript. Zunächst binden wir dazu ein paar Header ein.
Das Pragma `cling load ("libomp.so)` ermöglicht uns die Verwendung von OpenMP 

In [1]:
#include <stdbool.h>
#include <omp.h>
#include <stdlib.h>
#include <cstdio>
#pragma cling load("libomp.so")

Zunächst implementieren wir die Funktion, die wir Untersuchen wollen, in diesem Fall eine einfache Vektoraddition: 

In [2]:
//Ihre Funktion
void vector_add(double* a, double* b, double *c, int n) {
#pragma omp parallel for
for(int i = 0; i<n; i++) 
    c[i] = a[i]+b[i];
    }

In der folgenden Zelle können Sie nun Ihre Funktion testen. Damit wir im nächsten Schritt das automatische Benchmarking testen, sollten sie ihrem Code ein paar Kommentare hinzufügen:  

- Für Zeitmessungen muss der Code von `//start_main` und `//end_main` umgeschlossen sein, damit wird sichergestellt, dass der Code anschließend kompiliert wird. Dies sollte die "main" Funktion beschreiben. 

- Die Regionen, deren Laufzeit gemessen werden soll, müssen von `//start_timing` und `//end_timing` umgeschlossen werden, hier können sie die Gesamtlaufzeit messen. 

- Damit die Problemgröße verändert werden kann, sollte diese mit `int n= ;` definiert werden. Diese Zeile wird dann beim Benchmarking später ersetzt. 

Bitte beachten sie, dass sie in jedem Notebook nur *eine Zelle* zum testen/automatischen Benchmarking  verwenden sollten. Im zweiten Notebook über Profiling zeigen wir aber, wie sie mehrere Funktionen vergleichen können! 

Außerdem sollten sie die Anzahl der Threads hier nicht festlegen. Dies wird später automatisch festegelegt. Wenn sie hier z.B. `omp_set_num_threads()` verwenden würden, könnten sie damit ggf. die Werte, die durch das automatische Benchmarking gesetzt werden, überschreiben. 

Die Zelle wird nun einmal ausgeführt mit der gegebenen Problemgröße und einder Default-Anzahl an Threads. Dies ist hiflreich, um Fehler zu finden:

In [3]:
// start_main

int n = 1024*1024; //Hier n für die Problemgröße verwenden, damit es änderbar ist! 

#pragma omp parallel 
{
if(omp_get_thread_num()==0)
  printf("I have %d threads \n", omp_get_num_threads());
}


double *a = (double *) calloc(n, sizeof(double));
double *b = (double *) calloc(n, sizeof(double));
double *c = (double *) calloc(n, sizeof(double));

for(int i = 0; i<n; i++) {
 a[i] =  rand()%10; 
 b[i] = rand()%10;
}

// start_timing
vector_add(a,b,c,n);
//Funktionsaufruf

// end_timing



//überprüfung des Ergebnisses
for(int i=0;i<n; i++) {
   if(a[i]+b[i] -c[i] >= 1e-40)
       printf("error \n");

}



free(a);
free(b);
free(c);


// end_main

I have 64 threads 


Für die Laufzeitmessung, also das automatische Benchmarking,  brauchen wir eine Instanz der Klasse `performance`. Falls Sie den Name des Notebooks ändern oder ein neues Notebook verwenden,  sollte `vektoraddition_zeitmessung.ipynb` in der folgenden Zelle durch den Namen des Notebooks, in dem sie sich befinden,  erzetzt werden. Sie müssen also immer den Namen des aktuellen Notebooks verwenden. Zudem fügen wir den Header "performance.hpp" hinzu. Dies kann man theoretisch auch oben, bei den anderen Includes machen - allerdings führt dies bei MPI zu Problemen. Daher immer erst dann verwenden, wenn sie die Performance messen wollen. 

In [4]:
#include <performance.hpp>
performance p{"vektoraddition_zeitmessung.ipynb"};

Zum Anzeigen der Ergebnisse brauchen wir weiterhin Instanzen der Klasse `display::lazy_image`. Als Beispiel erzeugen wir hier zwei Instanzen: zur Visualisierung der Ergebnisse von Timing und Profiling. Auf das Proflig kommen wir im nächsten Schritt zu sprechen! 

In [5]:
display::lazy_image timingIm;

Die Laufzeit und der Speedup können durch einen Aufruf der Funktion `display_timing` gemessen/berechnet und visualisiert werden. Dieser Funktion können drei Parameter übergeben werden: 

- `num_threads`  Die Anzahl der Threads mit der die Funktion gestartet werden soll, Array {}
- `prob_size` Die Problemgröße ( werte, durch die `n ersetzt werden soll, Array {}
- `num_iter` Wie häufig soll die Messung  wiederholt werden

Die folgende Zelle zeigt ein Beispiel eines Aufrufs. Wenn sie die Funktion  `display_timing` für unser p-Objekt  ausführen, wird der Benchmark gestartet. Das kann, wenn sie sehr viele Parameter angegeben haben und größe Problemgrößen haben, etwas dauern. Sie werden nach/während der Ausführung einen neuen Ordner `results_` im Browser links entdecken, indem Zwischenergebnisse gespeichert werden. In den Log-Dateien können sie zudem den Fortschritt ihres Benchmarks verfolgen.

Die Klasse gibt Bilder mit den Daten zur Zeitmessung und für den Speedup zurrück. 

Der Aufruf des Bildes `timingImg` sorgt dafür, dass die Bilder in darunter angezeigt werden! 


In [6]:
timingIm = p.display_timing({1,2,4,8,16},{1000000,2000000,4000000,8000000,16000000}, 4);
timingIm

Parameter sets (Current set: prob_size=16000000, num_exec=16): 100%|██████████| 25/25 [00:23<00:00,  1.06it/s]


<!DOCTYPE html>
 
 
 
 timing_plot2

Wie wir nun sehen, ist der Speedup für 2 Threads und große Vektoren fast ideal (2), während bei mehr Threads der Overhead größer wird. 